<a href="https://colab.research.google.com/github/minuum/Liberty_ai/blob/Kobert_fine-tuning/KoBERT_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.말뭉치 학습(fine tuning) \
2.모델 업로드 \
2.hugging face 업로드 \
3.모델 개선(ex.전처리, 새로운 기법) \
4.답변 생성 및 평가


In [3]:
!pip install kobert-transformers
!pip install torch
!pip install transformers
!pip install sentencepiece

In [4]:
from kobert_transformers import get_kobert_model, get_tokenizer

# KoBERT 모델과 토크나이저 로드
model = get_kobert_model()
tokenizer = get_tokenizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# 입력 문장
text = "안녕하세요. 한국어 BERT 모델입니다."

# 입력 문장 토큰화
inputs = tokenizer(text, return_tensors="pt")

# 모델에 입력값 통과 (인퍼런스)
outputs = model(**inputs)

# 마지막 레이어의 출력값
last_hidden_states = outputs.last_hidden_state
print(last_hidden_states)


tensor([[[-0.0652, -0.0442,  0.0509,  ...,  0.0167, -0.0276, -0.0403],
         [-0.2190,  0.1401, -0.0050,  ...,  0.3916,  0.4374, -0.2533],
         [ 0.0158, -0.5050, -0.0610,  ...,  0.2848,  0.3007, -0.5008],
         ...,
         [ 0.0227,  0.1554, -0.0344,  ...,  0.0948,  0.2744, -0.0113],
         [ 0.0220, -0.3413,  0.0697,  ..., -0.2576,  0.3000, -0.3750],
         [-0.0280,  0.0093,  0.2115,  ..., -0.1682, -0.2065,  0.0197]]],
       grad_fn=<NativeLayerNormBackward0>)


In [6]:
from transformers import BertForSequenceClassification

# 분류 작업을 위한 KoBERT 모델 로드
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', num_labels=2)

# 분류할 문장
text = "이 문장은 긍정적입니다."

# 토큰화
inputs = tokenizer(text, return_tensors="pt")

# 모델 통과
outputs = model(**inputs)

# 로짓 값 (분류 결과)
logits = outputs.logits
print(logits)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.0968, -0.1286]], grad_fn=<AddmmBackward0>)


In [7]:
import torch

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 모델과 입력값을 GPU로 이동
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# 모델 실행
outputs = model(**inputs)


Using device: cpu


In [8]:
!pip install PyPDF2

In [9]:
import PyPDF2
file_path = '/content/drive/MyDrive/justice'
# 업로드한 PDF 파일 열기
file_path = '민법 선택조문 저장.pdf'  # 업로드한 파일 이름을 입력하세요.
with open(file_path, 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)

    # 모든 페이지의 텍스트 추출
    text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()

print(text)  # 추출된 텍스트 출력

FileNotFoundError: [Errno 2] No such file or directory: '민법 선택조문 저장.pdf'